# OpenAI Agents SDK - 동적 지시문과 모델 설정

이 튜토리얼은 OpenAI Agents SDK에서 에이전트의 동작을 런타임에 조정하는 방법을 다룬다. 동적 지시문을 통해 사용자 역할에 따라 에이전트의 성격과 전문성을 변경하고, ModelSettings를 통해 모델의 동작을 세밀하게 제어하는 방법을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 동적 지시문 에이전트 (Instruction Agent)

이 챕터에서는 런타임에 에이전트의 지시문을 동적으로 변경하는 방법을 다룬다. 동일한 에이전트가 사용자의 역할이나 상황에 따라 다른 전문가로 동작하게 할 수 있다.

### 학습 내용

- 사용자 역할에 따라 에이전트 동작을 동적으로 변경
- 다양한 컨텍스트에 맞게 에이전트의 성격과 전문성 적응
- 역할 기반 지시문 템플릿을 통한 특화된 응답
- 지시문 변경 시에도 대화 기록 유지

### 환경 설정

먼저 필요한 모듈을 임포트하고 환경 변수를 검증한다.

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 역할 기반 지시문 템플릿

사용자의 역할에 따라 다른 지시문을 반환하는 함수를 정의한다. 이 패턴을 사용하면 하나의 에이전트가 여러 전문가 역할을 수행할 수 있다. 학생에게는 튜터로, 개발자에게는 코딩 어시스턴트로, 비즈니스 담당자에게는 컨설턴트로, 크리에이터에게는 창의적 파트너로 동작한다.

In [2]:
def get_role_instructions(role: str) -> str:
    """
    사용자 역할에 따른 지시문을 반환하는 함수이다.
    
    Args:
        role: 사용자의 역할 (student, developer, business, creative)
    
    Returns:
        해당 역할에 맞는 지시문 문자열
    """
    instructions_map = {
        "student": "당신은 도움이 되는 튜터입니다. 개념을 명확하게 설명하고 예시를 제공하세요.",
        "developer": "당신은 코딩 어시스턴트입니다. 코드 예제와 기술적인 설명을 제공하세요.",
        "business": "당신은 비즈니스 컨설턴트입니다. 전략과 실용적인 조언에 집중하세요.",
        "creative": "당신은 창의적인 파트너입니다. 아이디어 브레인스토밍을 돕고 영감을 제공하세요."
    }
    return instructions_map.get(role, "당신은 도움이 되는 어시스턴트입니다.")

# 역할별 지시문 확인
print("=== 역할별 지시문 템플릿 ===")
for role in ["student", "developer", "business", "creative"]:
    print(f"\n[{role}]")
    print(f"  {get_role_instructions(role)}")

=== 역할별 지시문 템플릿 ===

[student]
  당신은 도움이 되는 튜터입니다. 개념을 명확하게 설명하고 예시를 제공하세요.

[developer]
  당신은 코딩 어시스턴트입니다. 코드 예제와 기술적인 설명을 제공하세요.

[business]
  당신은 비즈니스 컨설턴트입니다. 전략과 실용적인 조언에 집중하세요.

[creative]
  당신은 창의적인 파트너입니다. 아이디어 브레인스토밍을 돕고 영감을 제공하세요.


### 기본 에이전트 생성

동적으로 지시문을 변경할 기본 에이전트를 생성한다. 에이전트의 `instructions` 속성은 런타임에 직접 수정할 수 있다. 이 특성을 활용하여 동일한 에이전트 인스턴스가 다양한 역할을 수행하도록 할 수 있다.

In [3]:
# 기본 에이전트 생성
base_agent = Agent(
    name="DynamicAssistant",
    instructions="당신은 도움이 되는 어시스턴트입니다."
)

print(f"에이전트 '{base_agent.name}'가 생성되었다.")
print(f"초기 지시문: {base_agent.instructions}")

에이전트 'DynamicAssistant'가 생성되었다.
초기 지시문: 당신은 도움이 되는 어시스턴트입니다.


### 역할별 에이전트 테스트

각 역할에 대해 에이전트의 지시문을 동적으로 변경하고 대화를 수행한다. 역할마다 다른 질문을 사용하여 에이전트가 해당 역할에 맞는 전문적인 응답을 제공하는지 확인한다.

In [4]:
# 역할별 질문 정의
role_queries = {
    "student": "광합성이 어떻게 작동하는지 설명해 주시겠어요?",
    "developer": "이진 탐색 알고리즘을 어떻게 구현하나요?",
    "business": "시장 분석의 핵심 요소는 무엇인가요?",
    "creative": "새로운 마케팅 캠페인 아이디어가 필요해요."
}

print("=== 역할별 질문 ===")
for role, query in role_queries.items():
    print(f"[{role}] {query}")

=== 역할별 질문 ===
[student] 광합성이 어떻게 작동하는지 설명해 주시겠어요?
[developer] 이진 탐색 알고리즘을 어떻게 구현하나요?
[business] 시장 분석의 핵심 요소는 무엇인가요?
[creative] 새로운 마케팅 캠페인 아이디어가 필요해요.


In [5]:
async def test_role(role: str):
    """
    특정 역할에 대해 에이전트를 테스트하는 함수이다.
    
    Args:
        role: 테스트할 역할
    """
    print(f"\n{'='*50}")
    print(f"{role.upper()} 역할 테스트")
    print(f"{'='*50}")
    
    # 에이전트 지시문 동적 업데이트
    base_agent.instructions = get_role_instructions(role)
    print(f"지시문: {base_agent.instructions}")
    
    # 역할별 컨텍스트로 대화 시작
    conversation_history = [
        {"role": "user", "content": f"안녕하세요, 저는 {role}입니다. 도와주실 수 있나요?"}
    ]
    
    # 첫 번째 상호작용
    result1 = await Runner.run(base_agent, conversation_history)
    print(f"\n사용자: 안녕하세요, 저는 {role}입니다. 도와주실 수 있나요?")
    print(f"에이전트: {result1.final_output}")
    
    # 대화 기록 업데이트
    conversation_history = result1.to_input_list()
    
    # 두 번째 상호작용 - 역할별 전문 질문
    conversation_history.append({"role": "user", "content": role_queries[role]})
    result2 = await Runner.run(base_agent, conversation_history)
    
    print(f"\n사용자: {role_queries[role]}")
    print(f"에이전트: {result2.final_output}")
    
    # 대화 요약
    print(f"\n대화 요약:")
    print(f"   - 역할: {role}")
    print(f"   - 총 메시지 수: {len(result2.to_input_list())}")
    print(f"   - 사용된 토큰: {sum(r.usage.total_tokens for r in result2.raw_responses)}")
    
    return result2

In [6]:
# 학생 역할 테스트
await test_role("student")


STUDENT 역할 테스트
지시문: 당신은 도움이 되는 튜터입니다. 개념을 명확하게 설명하고 예시를 제공하세요.

사용자: 안녕하세요, 저는 student입니다. 도와주실 수 있나요?
에이전트: 안녕하세요, student님! 물론 도와드릴 수 있습니다. 궁금한 내용이나 공부하고 싶은 주제가 있으신가요? 어떤 도움이 필요한지 말씀해주시면, 최대한 쉽게 설명드리고 예시도 들어드리겠습니다. 편하게 질문해 주세요!

사용자: 광합성이 어떻게 작동하는지 설명해 주시겠어요?
에이전트: 네, 물론입니다!  
광합성(photosynthesis)은 식물, 조류, 일부 박테리아가 햇빛 에너지를 이용해 이산화탄소(CO₂)와 물(H₂O)로부터 포도당(C₆H₁₂O₆, 일종의 당)을 만들어내는 과정입니다. 쉽게 말해, 식물이 빛을 이용해 음식을 만드는 방법입니다. 광합성 덕분에 식물이 자라고, 우리가 호흡하는 산소도 만들어집니다.

### 광합성의 기본 과정

1. **재료**  
    - 이산화탄소 (CO₂): 식물의 잎에 있는 기공이라는 작은 구멍을 통해 들어옴  
    - 물 (H₂O): 뿌리를 통해 흡수함  
    - 빛 에너지: 태양에서 받아 잎에서 흡수함

2. **빛의 흡수**  
   잎의 세포에 있는 엽록체(Chloroplast)라는 기관에 엽록소(Chlorophyll)라는 초록색 색소가 있음. 이 엽록소가 태양빛 에너지를 흡수함.

3. **화학반응**
   빛 에너지를 사용해서 이산화탄소와 물로부터 포도당이라는 에너지원이 만들어지고, 산소(O₂)가 부산물로 나옴.

   광합성의 기본 식:
   ```
   6CO₂ + 6H₂O + 빛 에너지 → C₆H₁₂O₆ + 6O₂
   ```
   (이산화탄소 6분자 + 물 6분자 + 빛 에너지 → 포도당 1분자 + 산소 6분자)

### 예시로 쉽게 이해하기

만약 식물이 하나의 작은 "공장"이라고 가정해 봅시다.  
- **재료 창고(뿌리, 잎):** 물과 이산화탄소를 모은다.  
- **발전기(엽록소):** 빛 

RunResult(input=[{'role': 'user', 'content': '안녕하세요, 저는 student입니다. 도와주실 수 있나요?'}, {'id': 'msg_03a4119bb2c4e750006935908533ac81a081a1a39044e9d06d', 'content': [{'annotations': [], 'text': '안녕하세요, student님! 물론 도와드릴 수 있습니다. 궁금한 내용이나 공부하고 싶은 주제가 있으신가요? 어떤 도움이 필요한지 말씀해주시면, 최대한 쉽게 설명드리고 예시도 들어드리겠습니다. 편하게 질문해 주세요!', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': '광합성이 어떻게 작동하는지 설명해 주시겠어요?'}], new_items=[MessageOutputItem(agent=Agent(name='DynamicAssistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='당신은 도움이 되는 튜터입니다. 개념을 명확하게 설명하고 예시를 제공하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, re

In [7]:
# 개발자 역할 테스트
await test_role("developer")


DEVELOPER 역할 테스트
지시문: 당신은 코딩 어시스턴트입니다. 코드 예제와 기술적인 설명을 제공하세요.

사용자: 안녕하세요, 저는 developer입니다. 도와주실 수 있나요?
에이전트: 안녕하세요! 반갑습니다. 어떤 개발을 하고 계신가요?  
코드 작성, 버그 수정, 기술 설명, 프레임워크 선택 등 어떤 것이든 도와드릴 수 있습니다.  
궁금한 점이나 요청사항을 말씀해 주세요! 😊

사용자: 이진 탐색 알고리즘을 어떻게 구현하나요?
에이전트: 네! 이진 탐색(Binary Search)은 **정렬된 배열**에서 특정 값을 효율적으로 찾는 알고리즘입니다.  
중앙값과 비교해 절반씩 탐색 범위를 좁히는 방식입니다.

예시로 Python과 Java 두 가지 버전의 코드와, 만드는 데 필요한 핵심 개념을 설명해 드릴게요.

---

### 1. Python 구현

```python
def binary_search(arr, target):
    left, right = 0, len(arr) - 1

    while left <= right:
        mid = (left + right) // 2  # 중앙 인덱스 계산
        if arr[mid] == target:
            return mid  # 목표값을 찾은 경우
        elif arr[mid] < target:
            left = mid + 1  # 오른쪽 구간 탐색
        else:
            right = mid - 1  # 왼쪽 구간 탐색

    return -1  # 목표값이 없는 경우
```

#### 사용 예시

```python
arr = [1, 3, 5, 7, 9, 11, 13, 15]
target = 7
result = binary_search(arr, target)  # 결과: 3 (arr[3] == 7)
```

---

### 2. Java 구현

```java
public class BinarySearch {


RunResult(input=[{'role': 'user', 'content': '안녕하세요, 저는 developer입니다. 도와주실 수 있나요?'}, {'id': 'msg_0f58da154797f10f00693590916fec819393147f007f79e4ef', 'content': [{'annotations': [], 'text': '안녕하세요! 반갑습니다. 어떤 개발을 하고 계신가요?  \n코드 작성, 버그 수정, 기술 설명, 프레임워크 선택 등 어떤 것이든 도와드릴 수 있습니다.  \n궁금한 점이나 요청사항을 말씀해 주세요! 😊', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': '이진 탐색 알고리즘을 어떻게 구현하나요?'}], new_items=[MessageOutputItem(agent=Agent(name='DynamicAssistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='당신은 코딩 어시스턴트입니다. 코드 예제와 기술적인 설명을 제공하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_inclu

In [8]:
# 비즈니스 역할 테스트
await test_role("business")


BUSINESS 역할 테스트
지시문: 당신은 비즈니스 컨설턴트입니다. 전략과 실용적인 조언에 집중하세요.

사용자: 안녕하세요, 저는 business입니다. 도와주실 수 있나요?
에이전트: 안녕하세요! 비즈니스 컨설턴트로서 도와드릴 수 있습니다.  
궁금하신 사항, 현재 겪고 있는 이슈, 또는 개선하고 싶은 부분 등에 대해 말씀해 주시면 전략적이고 실질적인 조언을 드릴 수 있습니다.

예를 들어:
- 신규 사업 아이디어 검토
- 매출 또는 수익성 개선 전략
- 마케팅 및 브랜드 전략
- 조직/인력 관리 방안
- 비용 절감 방안
- 투자 유치 자료 점검 등

궁금한 주제나 상황을 조금 더 구체적으로 말씀해 주시면 맞춤형 조언을 드리겠습니다. 어떤 부분에서 도움을 원하시나요?

사용자: 시장 분석의 핵심 요소는 무엇인가요?
에이전트: 좋은 질문입니다. **시장 분석(Market Analysis)**은 전략 수립과 의사결정의 기본이 되는 단계로, 다음과 같은 핵심 요소들로 구성됩니다.

---

## 시장 분석의 핵심 요소

1. **시장 규모(Market Size)**
   - 전체 시장(또는 세그먼트)의 현재 및 잠재적 수요, 매출, 판매량 등 규모를 수치로 파악
   - TAM(Total Addressable Market), SAM(Serviceable Available Market), SOM(Serviceable Obtainable Market) 개념 활용

2. **시장 성장률(Market Growth Rate) 및 트렌드**
   - 시장이 연 평균 얼마나 성장하고 있는지, 미래 성장 전망은 어떤지 분석
   - 성장에 영향을 미치는 거시적 요인(인구, 정책, 기술 변화 등) 파악

3. **고객 분석(Customer Analysis·Target Customer)**
   - 주요 고객층(페르소나), 고객의 니즈, 구매 행동, 지불 의사, 가치 기준 등
   - B2C와 B2B 비즈니스 각각의 고객 세분화

4. **경쟁사 분석(Competitive A

RunResult(input=[{'role': 'user', 'content': '안녕하세요, 저는 business입니다. 도와주실 수 있나요?'}, {'id': 'msg_0c2c729b0adc0ed5006935909c4ad8819fa79efb3821b307ab', 'content': [{'annotations': [], 'text': '안녕하세요! 비즈니스 컨설턴트로서 도와드릴 수 있습니다.  \n궁금하신 사항, 현재 겪고 있는 이슈, 또는 개선하고 싶은 부분 등에 대해 말씀해 주시면 전략적이고 실질적인 조언을 드릴 수 있습니다.\n\n예를 들어:\n- 신규 사업 아이디어 검토\n- 매출 또는 수익성 개선 전략\n- 마케팅 및 브랜드 전략\n- 조직/인력 관리 방안\n- 비용 절감 방안\n- 투자 유치 자료 점검 등\n\n궁금한 주제나 상황을 조금 더 구체적으로 말씀해 주시면 맞춤형 조언을 드리겠습니다. 어떤 부분에서 도움을 원하시나요?', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': '시장 분석의 핵심 요소는 무엇인가요?'}], new_items=[MessageOutputItem(agent=Agent(name='DynamicAssistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='당신은 비즈니스 컨설턴트입니다. 전략과 실용적인 조언에 집중하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_

In [9]:
# 크리에이티브 역할 테스트
await test_role("creative")


CREATIVE 역할 테스트
지시문: 당신은 창의적인 파트너입니다. 아이디어 브레인스토밍을 돕고 영감을 제공하세요.

사용자: 안녕하세요, 저는 creative입니다. 도와주실 수 있나요?
에이전트: 안녕하세요, creative님! 만나서 반갑습니다. 물론이죠, 어떤 아이디어가 필요하신가요?  
프로젝트의 주제나 목적, 관심 있는 분야, 또는 고민하고 있는 부분을 알려주시면 더 맞춤형으로 도와드릴 수 있어요.  
브레인스토밍은 어떤 방식이든 환영입니다: 자유 연상, 마인드맵, 리스트업, 콜라주 등등.  
시작을 위해 간단히 설명해 주시겠어요? 😊

사용자: 새로운 마케팅 캠페인 아이디어가 필요해요.
에이전트: 좋아요! 새로운 마케팅 캠페인이 필요하군요. 몇 가지 확인 질문 드릴게요—답변해주시면 더 맞춤형 아이디어를 드릴 수 있습니다.

1. **브랜드/제품/서비스**: 어떤 것을 홍보하시나요?
2. **타겟 고객층**: 주로 누구에게 어필하고 싶으세요? (연령, 성별, 관심사 등)
3. **캠페인 목적**: 인지도 향상, 판매 증대, 브랜드 이미지 개선 등
4. **진행 예정 채널**: SNS, 오프라인, 홈페이지, 이메일 등
5. **특별히 고려해야 할 부분**: 예산, 일정, 피해야 할 컨셉, 핵심 메시지 등

일단 정보 없이도 쓸 수 있는 **보편적인 캠페인 컨셉 3가지**를 먼저 드릴게요:

---

### 1. 사용자 참여형 캠페인 (UGC, User Generated Content)
- 고객들이 제품/서비스와 관련된 콘텐츠(사진, 영상, 후기 등)를 직접 만들어 공유하게 합니다.
- 예: 인스타그램 해시태그 챌린지, “내가 만든 OOO 활용법” 영상 공모전 등
- 장점: 자연스러운 바이럴, 참여자와 소통 강화, 저비용

### 2. 스토리텔링 인터랙티브 캠페인
- 브랜드 또는 제품이 고객의 삶을 어떻게 변화시키는지 ‘사연’으로 전달.
- 마이크로 웹사이트에서 퀴즈, 선택형 이야기(“내가 주인공이라면” 등) 제공 후 맞춤형 쿠폰 지급
- 장점: 몰입

RunResult(input=[{'role': 'user', 'content': '안녕하세요, 저는 creative입니다. 도와주실 수 있나요?'}, {'id': 'msg_0b3546e00917511f00693590a904e8819f8c9e67453a871b18', 'content': [{'annotations': [], 'text': '안녕하세요, creative님! 만나서 반갑습니다. 물론이죠, 어떤 아이디어가 필요하신가요?  \n프로젝트의 주제나 목적, 관심 있는 분야, 또는 고민하고 있는 부분을 알려주시면 더 맞춤형으로 도와드릴 수 있어요.  \n브레인스토밍은 어떤 방식이든 환영입니다: 자유 연상, 마인드맵, 리스트업, 콜라주 등등.  \n시작을 위해 간단히 설명해 주시겠어요? 😊', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': '새로운 마케팅 캠페인 아이디어가 필요해요.'}], new_items=[MessageOutputItem(agent=Agent(name='DynamicAssistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='당신은 창의적인 파트너입니다. 아이디어 브레인스토밍을 돕고 영감을 제공하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None,

### 동적 지시문의 활용 사례

동적 지시문 패턴은 다양한 상황에서 활용할 수 있다. 사용자 역할 외에도 시간대, 언어, 난이도 수준 등에 따라 에이전트의 동작을 조정할 수 있다.

In [10]:
# 난이도 기반 지시문 예제
def get_difficulty_instructions(level: str) -> str:
    """
    난이도 수준에 따른 지시문을 반환하는 함수이다.
    """
    instructions_map = {
        "beginner": """당신은 초보자를 위한 친절한 교사입니다.
        - 전문 용어를 피하고 쉬운 말로 설명하세요
        - 일상적인 비유를 많이 사용하세요
        - 단계별로 천천히 설명하세요""",
        
        "intermediate": """당신은 중급자를 위한 멘토입니다.
        - 기본 개념은 알고 있다고 가정하세요
        - 실용적인 예제와 응용을 강조하세요
        - 관련 개념을 연결지어 설명하세요""",
        
        "advanced": """당신은 고급 전문가를 위한 컨설턴트입니다.
        - 심층적인 기술적 세부사항을 제공하세요
        - 최신 트렌드와 모범 사례를 언급하세요
        - 복잡한 시나리오와 엣지 케이스를 다루세요"""
    }
    return instructions_map.get(level, instructions_map["intermediate"])

print("=== 난이도별 지시문 예제 ===")
for level in ["beginner", "intermediate", "advanced"]:
    print(f"\n[{level}]")
    print(get_difficulty_instructions(level))

=== 난이도별 지시문 예제 ===

[beginner]
당신은 초보자를 위한 친절한 교사입니다.
        - 전문 용어를 피하고 쉬운 말로 설명하세요
        - 일상적인 비유를 많이 사용하세요
        - 단계별로 천천히 설명하세요

[intermediate]
당신은 중급자를 위한 멘토입니다.
        - 기본 개념은 알고 있다고 가정하세요
        - 실용적인 예제와 응용을 강조하세요
        - 관련 개념을 연결지어 설명하세요

[advanced]
당신은 고급 전문가를 위한 컨설턴트입니다.
        - 심층적인 기술적 세부사항을 제공하세요
        - 최신 트렌드와 모범 사례를 언급하세요
        - 복잡한 시나리오와 엣지 케이스를 다루세요


In [11]:
# 언어/스타일 기반 지시문 예제
def get_style_instructions(style: str) -> str:
    """
    응답 스타일에 따른 지시문을 반환하는 함수이다.
    """
    instructions_map = {
        "formal": "격식체를 사용하여 전문적이고 정중하게 응답하세요.",
        "casual": "편안하고 친근한 말투로 대화하듯이 응답하세요.",
        "concise": "핵심만 간결하게 요약하여 응답하세요. 불필요한 설명은 생략하세요.",
        "detailed": "상세하고 포괄적으로 설명하세요. 예시와 배경 정보를 풍부하게 포함하세요."
    }
    return instructions_map.get(style, instructions_map["casual"])

print("=== 스타일별 지시문 예제 ===")
for style in ["formal", "casual", "concise", "detailed"]:
    print(f"[{style}] {get_style_instructions(style)}")

=== 스타일별 지시문 예제 ===
[formal] 격식체를 사용하여 전문적이고 정중하게 응답하세요.
[casual] 편안하고 친근한 말투로 대화하듯이 응답하세요.
[concise] 핵심만 간결하게 요약하여 응답하세요. 불필요한 설명은 생략하세요.
[detailed] 상세하고 포괄적으로 설명하세요. 예시와 배경 정보를 풍부하게 포함하세요.


---

## 2. 모델 설정 구성

이 챕터에서는 `ModelSettings`를 사용하여 에이전트의 동작을 세밀하게 조정하는 방법을 다룬다. temperature, top_p, 토큰 제한 등의 파라미터를 조정하여 다양한 사용 사례에 최적화된 에이전트를 구성할 수 있다.

### 학습 내용

- 다양한 사용 사례에 맞는 temperature, top_p, 토큰 제한 구성
- 창의적 작업 vs 분석적 작업 vs 대화형 작업을 위한 에이전트 구성
- 모델 파라미터가 에이전트 응답에 미치는 영향 이해
- 비용, 속도, 품질에 기반한 최적화

### ModelSettings 파라미터 이해

`ModelSettings`는 LLM의 동작을 제어하는 다양한 파라미터를 제공한다. 각 파라미터의 역할을 이해하면 목적에 맞는 에이전트를 효과적으로 구성할 수 있다.

| 파라미터 | 설명 | 범위 | 효과 |
|---------|------|------|------|
| temperature | 응답의 무작위성/창의성 | 0.0-2.0 | 높을수록 다양하고 창의적 |
| top_p | 확률 질량의 상위 p% 사용 | 0.0-1.0 | 낮을수록 집중된 응답 |
| max_tokens | 최대 응답 토큰 수 | 1-모델한계 | 응답 길이 제한 |
| frequency_penalty | 반복 단어 억제 | -2.0-2.0 | 높을수록 반복 감소 |
| presence_penalty | 새로운 주제 장려 | -2.0-2.0 | 높을수록 다양한 주제 |

### 창의적 작업용 에이전트

창의적인 글쓰기나 아이디어 생성 작업에는 높은 temperature와 top_p를 사용한다. 이렇게 하면 에이전트가 더 다양하고 예상치 못한 응답을 생성한다. `frequency_penalty`와 `presence_penalty`를 적절히 설정하여 반복을 줄이고 다양한 어휘를 사용하도록 유도한다.

In [13]:
# 창의적 작업용 에이전트 구성
creative_agent = Agent(
    name="CreativeWriter",
    model=MODEL,
    model_settings=ModelSettings(
        temperature=0.8,        # 높은 창의성과 변화
        top_p=0.9,             # 상위 90% 확률 질량 고려
        max_tokens=1500,       # 긴 응답 허용
        frequency_penalty=0.1,  # 반복에 대한 약한 페널티
        presence_penalty=0.1    # 다양한 어휘 장려
    ),
    instructions="당신은 매력적인 서사를 전문으로 하는 창의적인 작가입니다."
)

print("=== 창의적 작업용 에이전트 ===")
print(f"이름: {creative_agent.name}")
print(f"모델: {creative_agent.model}")
print(f"temperature: 0.8 (높은 창의성)")
print(f"top_p: 0.9 (넓은 확률 분포)")
print(f"max_tokens: 1500 (긴 응답)")

=== 창의적 작업용 에이전트 ===
이름: CreativeWriter
모델: gpt-4o-mini
temperature: 0.8 (높은 창의성)
top_p: 0.9 (넓은 확률 분포)
max_tokens: 1500 (긴 응답)


In [14]:
# 창의적 에이전트 테스트
async def test_creative_agent():
    result = await Runner.run(
        creative_agent, 
        "우주를 여행하는 고양이에 대한 짧은 이야기의 도입부를 써주세요."
    )
    print("\n[창의적 에이전트 응답]")
    print(result.final_output)
    print(f"\n사용된 토큰: {sum(r.usage.total_tokens for r in result.raw_responses)}")

await test_creative_agent()


[창의적 에이전트 응답]
한때 평범한 고양이로 살던 루나는 어느 날 창가에 앉아 별을 바라보다가, 우주에 대한 호기심이 점점 커지기 시작했다. 그녀의 눈동자는 반짝이는 별빛처럼 빛났고, 매일 밤마다 꿈속에서 별들 사이를 유영하는 모습을 그렸다. 그러던 중, 그녀는 고양이 우주선을 발견하게 된다. 이 우주선은 마치 고양이의 숨결처럼 부드럽고, 별빛을 흡수하는 듯한 은은한 광채를 뿜어냈다. 루나는 주인 없는 우주선에 뛰어올라, 그녀의 모험이 시작되는 순간을 기다렸다. 

"별들을 향해!" 루나는 외쳤고, 우주선은 드디어 고양이의 소원을 들어주기 위해 하늘로 솟구쳤다.

사용된 토큰: 250


### 분석적 작업용 에이전트

데이터 분석이나 사실 기반 작업에는 낮은 temperature를 사용한다. 이렇게 하면 에이전트가 더 일관적이고 예측 가능한 응답을 생성한다. 반복 페널티는 0으로 설정하여 필요한 경우 동일한 용어를 반복할 수 있게 한다.

In [15]:
# 분석적 작업용 에이전트 구성
analytical_agent = Agent(
    name="DataAnalyst", 
    model=MODEL,
    model_settings=ModelSettings(
        temperature=0.1,        # 낮은 창의성, 높은 일관성
        top_p=0.5,             # 집중된 확률 분포
        max_tokens=800,        # 간결한 응답
        frequency_penalty=0.0,  # 반복 페널티 없음
        presence_penalty=0.0    # 어휘 다양성 압력 없음
    ),
    instructions="당신은 정확하고 객관적인 분석을 제공하는 데이터 분석가입니다."
)

print("=== 분석적 작업용 에이전트 ===")
print(f"이름: {analytical_agent.name}")
print(f"모델: {analytical_agent.model}")
print(f"temperature: 0.1 (높은 일관성)")
print(f"top_p: 0.5 (집중된 분포)")
print(f"max_tokens: 800 (간결한 응답)")

=== 분석적 작업용 에이전트 ===
이름: DataAnalyst
모델: gpt-4o-mini
temperature: 0.1 (높은 일관성)
top_p: 0.5 (집중된 분포)
max_tokens: 800 (간결한 응답)


In [16]:
# 분석적 에이전트 테스트
async def test_analytical_agent():
    result = await Runner.run(
        analytical_agent, 
        "A/B 테스트의 통계적 유의성을 판단하는 방법을 설명해주세요."
    )
    print("\n[분석적 에이전트 응답]")
    print(result.final_output)
    print(f"\n사용된 토큰: {sum(r.usage.total_tokens for r in result.raw_responses)}")

await test_analytical_agent()


[분석적 에이전트 응답]
A/B 테스트의 통계적 유의성을 판단하는 방법은 다음과 같은 단계로 진행됩니다:

1. **가설 설정**:
   - **귀무가설(H0)**: 두 그룹 간에 차이가 없다.
   - **대립가설(H1)**: 두 그룹 간에 차이가 있다.

2. **데이터 수집**:
   - A 그룹(통제 그룹)과 B 그룹(실험 그룹)에서 필요한 데이터를 수집합니다. 각 그룹의 샘플 크기와 성과 지표(예: 클릭률, 전환율 등)를 기록합니다.

3. **적절한 통계 검정 선택**:
   - 데이터의 특성과 분포에 따라 적절한 통계 검정을 선택합니다. 일반적으로 사용되는 검정 방법은 다음과 같습니다:
     - **t-검정**: 두 그룹의 평균을 비교할 때 사용.
     - **카이제곱 검정**: 범주형 데이터의 비율을 비교할 때 사용.
     - **Z-검정**: 샘플 크기가 클 때 평균 비교에 사용.

4. **유의수준 설정**:
   - 일반적으로 0.05(5%)의 유의수준을 설정합니다. 이는 귀무가설이 참일 때, 5%의 확률로 잘못된 결론을 내릴 수 있음을 의미합니다.

5. **검정 통계량 계산**:
   - 선택한 통계 검정 방법에 따라 검정 통계량을 계산합니다. 예를 들어, t-검정의 경우 t-값을 계산합니다.

6. **p-값 계산**:
   - 검정 통계량을 기반으로 p-값을 계산합니다. p-값은 귀무가설이 참일 때 관찰된 데이터가 나타날 확률을 나타냅니다.

7. **결과 해석**:
   - p-값이 설정한 유의수준(예: 0.05)보다 작으면 귀무가설을 기각하고, 두 그룹 간에 유의미한 차이가 있다고 결론 내립니다. 반대로 p-값이 유의수준보다 크면 귀무가설을 기각하지 않습니다.

8. **결과 보고**:
   - A/B 테스트의 결과를 명확하게 보고하고, 실험의 맥락과 함께 해석합니다. 필요한 경우 신뢰 구간을 제공하여 결과의 신뢰성을 강조할 수 있습니다.

이 과정을 통해 A/B 테스트의 통계적 유의성을 판단할 수 있습니다.

사용된

### 대화형 에이전트

실시간 채팅에는 균형 잡힌 설정과 빠른 응답이 중요하다. `gpt-4o-mini`와 같은 더 빠르고 비용 효율적인 모델을 사용하고, max_tokens를 낮게 설정하여 짧은 응답을 유도한다. temperature는 중간 값으로 설정하여 자연스러우면서도 예측 가능한 대화를 만든다.

In [17]:
# 대화형 에이전트 구성
conversational_agent = Agent(
    name="ChatAgent",
    model=MODEL,      # 채팅에 더 빠르고 비용 효율적
    model_settings=ModelSettings(
        temperature=0.5,        # 균형 잡힌 창의성과 일관성
        top_p=0.8,             # 다양성과 집중의 좋은 균형
        max_tokens=300         # 채팅을 위한 짧은 응답
    ),
    instructions="당신은 친근하고 대화적인 어시스턴트입니다."
)

print("=== 대화형 에이전트 ===")
print(f"이름: {conversational_agent.name}")
print(f"모델: {conversational_agent.model} (빠르고 경제적)")
print(f"temperature: 0.5 (균형 잡힌)")
print(f"top_p: 0.8 (적당한 다양성)")
print(f"max_tokens: 300 (짧은 응답)")

=== 대화형 에이전트 ===
이름: ChatAgent
모델: gpt-4o-mini (빠르고 경제적)
temperature: 0.5 (균형 잡힌)
top_p: 0.8 (적당한 다양성)
max_tokens: 300 (짧은 응답)


In [18]:
# 대화형 에이전트 테스트
async def test_conversational_agent():
    result = await Runner.run(
        conversational_agent, 
        "안녕하세요! 오늘 날씨가 좋네요. 뭐 하고 계세요?"
    )
    print("\n[대화형 에이전트 응답]")
    print(result.final_output)
    print(f"\n사용된 토큰: {sum(r.usage.total_tokens for r in result.raw_responses)}")

await test_conversational_agent()


[대화형 에이전트 응답]
안녕하세요! 날씨가 좋다니 기쁘네요. 저는 여러분과 대화하고 정보를 나누고 있어요. 오늘 특별한 계획이 있으신가요?

사용된 토큰: 79


### Temperature 효과 비교 실험

동일한 프롬프트에 대해 서로 다른 temperature 값을 적용하여 응답의 차이를 관찰한다.

In [20]:
async def compare_temperatures():
    """다양한 temperature 값의 효과를 비교하는 함수이다."""
    
    prompt = "바다에 대한 은유적 표현을 하나 만들어주세요."
    temperatures = [0.1, 0.5, 0.9]
    
    print("=== Temperature 효과 비교 ===")
    print(f"프롬프트: {prompt}\n")
    
    for temp in temperatures:
        agent = Agent(
            name=f"TempAgent_{temp}",
            model="gpt-4o-mini",
            model_settings=ModelSettings(
                temperature=temp,
                max_tokens=100
            ),
            instructions="창의적인 문장을 만들어주세요."
        )
        
        result = await Runner.run(agent, prompt)
        print(f"[temperature={temp}]")
        print(f"  {result.final_output}")
        print()

await compare_temperatures()

=== Temperature 효과 비교 ===
프롬프트: 바다에 대한 은유적 표현을 하나 만들어주세요.

[temperature=0.1]
  바다는 마음의 깊이를 담은 거울, 파도가 일렁일 때마다 감정의 비밀을 속삭이는 곳이다.

[temperature=0.5]
  바다는 마음의 거울, 깊은 감정이 파도처럼 밀려오는 곳이다.

[temperature=0.9]
  바다는 마음의 거울, 무수한 감정의 파도가 넘실대는 곳이다.



---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 동적 지시문과 모델 설정을 다루었다. 다음과 같은 내용을 학습하였다:

1. **동적 지시문**: 런타임에 에이전트의 역할과 동작을 변경하는 방법
2. **역할 기반 템플릿**: 사용자 역할에 따른 전문화된 응답 제공
3. **ModelSettings 파라미터**: temperature, top_p, max_tokens 등의 효과 이해
4. **사용 사례별 구성**: 창의적, 분석적, 대화형 작업에 맞는 최적 설정
5. **비용 최적화**: 품질과 비용의 균형을 맞추는 전략

이러한 패턴을 적용하면 다양한 사용 사례와 사용자 유형에 적응하는 유연하고 효율적인 에이전트 시스템을 구축할 수 있다.